In [1]:
import numpy as np
import pandas as pd
import geonamescache 
import unicodedata
import unidecode
import re
import multiprocessing as mp
import os
from multiprocessing import Pool, TimeoutError
from pandas.io.json import json_normalize

gc = geonamescache.GeonamesCache()

# read data from file
df = pd.read_csv('./data/headlines.txt', sep="\n", header=None, names=['headline'])

# get cities from geonamescache and sort them by string-length descending
cities = gc.get_cities()
known_cities=list(map(lambda k: cities[k]['name'], cities)) 
known_cities=list(map(lambda v: unidecode.unidecode(v), known_cities))
known_cities.sort(key=lambda k: len(k),reverse=True)

# get countries in same way
countries = gc.get_countries()
known_countries=list(map(lambda k: countries[k]['name'], countries))
known_countries=list(map(lambda v: unidecode.unidecode(v), known_countries))
known_countries.sort(key=lambda k: len(k),reverse=True)

# df is a working data frame
df = df.assign(cities=None)
df['cities'] =  [[] for _ in range(len(df))]
df['countries'] =  [[] for _ in range(len(df))]

print( 'number of cities = ', len(known_cities))
print( 'number of countries = ', len(known_countries))

list_of_cities=list(map(str, known_cities))
list_of_countries=list(map(str, known_countries))

city_pattern=re.compile('|'.join(map(lambda x: '\\b' + re.escape(x) + '\\b', list_of_cities)),flags=re.IGNORECASE+re.MULTILINE)
country_pattern=re.compile('|'.join(map(lambda x: '\\b' + re.escape(x) + '\\b', list_of_countries)),flags=re.IGNORECASE+re.MULTILINE)

for index, row in df.iterrows(): 
    items = re.findall( city_pattern, row['headline'] )
    for item in items:
        row['cities'].append(item)
    items = re.findall( country_pattern, row['headline'] )
    for item in items:
        row['countries'].append(item)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

df['city']='-'
df['country']='-'

for index, row in df.iterrows():
    list_of_cities = row['cities']
    if len(list_of_cities) > 0:
        list_of_cities.sort(key=lambda k: len(k), reverse=True)
        row['city'] = list_of_cities[0]
    list_of_countries = row['countries']
    if len(list_of_countries) > 0:
        list_of_countries.sort(key=lambda k: len(k), reverse=True)
        row['country'] = list_of_countries[0]
    current_city = row['city']
    current_country= row['country']
    if current_country != '-':
        tmp = gc.get_countries_by_names()
        print(tmp)
    # todo
    #city_matches = gc.get_cities_by_name( row['city'])
    #if len(city_matches) > 1:
    #    print( current_city, ' ## ', city_matches )
    
country_df = pd.DataFrame.from_dict( gc.get_countries_by_names(),orient='index')    
country_df


number of cities =  24336
number of countries =  252
{'Andorra': {'geonameid': 3041565, 'name': 'Andorra', 'iso': 'AD', 'iso3': 'AND', 'isonumeric': 20, 'fips': 'AN', 'continentcode': 'EU', 'capital': 'Andorra la Vella', 'areakm2': 468, 'population': 84000, 'tld': '.ad', 'currencycode': 'EUR', 'currencyname': 'Euro', 'phone': '376', 'postalcoderegex': '^(?:AD)*(\\d{3})$', 'languages': 'ca', 'neighbours': 'ES,FR'}, 'United Arab Emirates': {'geonameid': 290557, 'name': 'United Arab Emirates', 'iso': 'AE', 'iso3': 'ARE', 'isonumeric': 784, 'fips': 'AE', 'continentcode': 'AS', 'capital': 'Abu Dhabi', 'areakm2': 82880, 'population': 4975593, 'tld': '.ae', 'currencycode': 'AED', 'currencyname': 'Dirham', 'phone': '971', 'postalcoderegex': '', 'languages': 'ar-AE,fa,en,hi,ur', 'neighbours': 'SA,OM'}, 'Afghanistan': {'geonameid': 1149361, 'name': 'Afghanistan', 'iso': 'AF', 'iso3': 'AFG', 'isonumeric': 4, 'fips': 'AF', 'continentcode': 'AS', 'capital': 'Kabul', 'areakm2': 647500, 'population':

,geonameid,name,iso,iso3,isonumeric,fips,continentcode,capital,areakm2,population,tld,currencycode,currencyname,phone,postalcoderegex,languages,neighbours
Andorra,3041565,Andorra,AD,AND,20,AN,EU,Andorra la Vella,468,84000,.ad,EUR,Euro,376,^(?:AD)*(\d{3})$,ca,"ES,FR"
United Arab Emirates,290557,United Arab Emirates,AE,ARE,784,AE,AS,Abu Dhabi,82880,4975593,.ae,AED,Dirham,971,,"ar-AE,fa,en,hi,ur","SA,OM"
Afghanistan,1149361,Afghanistan,AF,AFG,4,AF,AS,Kabul,647500,29121286,.af,AFN,Afghani,93,,"fa-AF,ps,uz-AF,tk","TM,CN,IR,TJ,PK,UZ"
Antigua and Barbuda,3576396,Antigua and Barbuda,AG,ATG,28,AC,NA,St. John's,443,86754,.ag,XCD,Dollar,+1-268,,en-AG,
Anguilla,3573511,Anguilla,AI,AIA,660,AV,NA,The Valley,102,13254,.ai,XCD,Dollar,+1-264,,en-AI,
Albania,783754,Albania,AL,ALB,8,AL,EU,Tirana,28748,2986952,.al,ALL,Lek,355,^(\d{4})$,"sq,el","MK,GR,ME,RS,XK"
Armenia,174982,Armenia,AM,ARM,51,AM,AS,Yerevan,29800,2968000,.am,AMD,Dram,374,^(\d{6})$,hy,"GE,IR,AZ,TR"
Angola,3351879,Angola,AO,AGO,24,AO,AF,Luanda,1246700,13068161,.ao,AOA,Kwanza,244,,pt-AO,"CD,NA,ZM,CG"
Antarctica,6697173,Antarctica,AQ,ATA,10,AY,AN,,14000000,0,.aq,,,,,,
Argentina,3865483,Argentina,AR,ARG,32,AR,SA,Buenos Aires,2766890,41343201,.ar,ARS,Peso,54,"^[A-Z]?\d{4}[A-Z]{0,3}$","es-AR,en,it,de,fr,gn","CL,BO,UY,PY,BR"
